In [1]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import json, os, csv, shutil
from glob import glob
import matplotlib.pyplot as plt

In [2]:
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Lambda, Activation, Flatten, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop, SGD
from keras.utils import np_utils

import h5py
import cv2
import PIL
from PIL import Image

Using TensorFlow backend.


In [3]:
path = 'data/'

In [4]:
batch_size = 24

# Data preprocessing

In [5]:
dir_names = os.listdir(os.path.join(path, 'train/'))
dir_names

['YFT', 'ALB', 'NoF', 'SHARK', 'BET', 'DOL', 'LAG', 'OTHER']

In [6]:
dir_dict = {}

for i in range(len(dir_names)):
    dir_dict[dir_names[i]] = i

In [7]:
dir_dict

{'ALB': 1,
 'BET': 4,
 'DOL': 5,
 'LAG': 6,
 'NoF': 2,
 'OTHER': 7,
 'SHARK': 3,
 'YFT': 0}

In [8]:
X_train = []
y_train = []
trn_size = []
trn_filename = []

In [9]:
for folder in os.listdir(path + 'train/'):
    for file in os.listdir(path + 'train/' + folder + '/'):
        if os.path.isfile(path + 'train/' + folder + '/' + file):
            
            input_img = cv2.imread(path + 'train/' + folder + '/' + file)
            height, width, channel = input_img.shape
            input_img = cv2.resize(input_img, (224, 224))
            X_train.append(input_img)
            
            y_cat = np_utils.to_categorical(dir_dict[folder], 8) # 8 represent number of class
            y_train.append(y_cat)
            
            trn_size.append([width, height])
            trn_filename.append(file)

In [10]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [11]:
print(X_train.shape)
print(y_train.shape)
print(trn_size[0:5])
print(trn_filename[0:5])

(3025, 224, 224, 3)
(3025, 8)
[[1280, 720], [1280, 750], [1280, 974], [1280, 720], [1280, 974]]
['img_04942.jpg', 'img_05676.jpg', 'img_01087.jpg', 'img_00436.jpg', 'img_07233.jpg']


In [12]:
y_train[466]

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [13]:
X_valid = []
y_valid = []
val_size = []
val_filename = []

In [14]:
for folder in os.listdir(path + 'valid/'):
    for file in os.listdir(path + 'valid/' + folder + '/'):
        if os.path.isfile(path + 'valid/' + folder + '/' + file):
            
            input_img = cv2.imread(path + 'valid/' + folder + '/' + file)
            height, width, channel = input_img.shape
            input_img = cv2.resize(input_img, (224, 224))
            X_valid.append(input_img)
            
            y_cat = np_utils.to_categorical(dir_dict[folder], 8) # 8 represent number of class
            y_valid.append(y_cat)
            
            val_size.append([width, height])
            val_filename.append(file)

In [15]:
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)

In [16]:
print(X_valid.shape)
print(y_valid.shape)
print(val_size[0:5])
print(val_filename[0:5])

(752, 224, 224, 3)
(752, 8)
[[1280, 720], [1280, 750], [1280, 974], [1280, 720], [1280, 720]]
['img_00806.jpg', 'img_07624.jpg', 'img_03372.jpg', 'img_04825.jpg', 'img_02075.jpg']


In [17]:
y_valid[0]

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

# BB

In [18]:
import ujson as json

In [19]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']

In [20]:
bb_json = {}

for c in anno_classes:
    j = json.load(open('{}annos/{}_labels.json'.format(path, c), 'r'))
    
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations']) > 0:
            bb_json[l['filename']] = l['annotations'][-1]

In [21]:
bb_json['img_07763.jpg']

{'class': 'rect',
 'height': 127.00000000000045,
 'width': 121.00000000000045,
 'x': 636.0000000000023,
 'y': 353.00000000000125}

In [22]:
empty_bbox = {'height': 0., 'width': 0., 'x': 0., 'y': 0.}

In [23]:
for f in trn_filename:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox
for f in val_filename:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox

In [24]:
trn_resize_dim = []
val_resize_dim = []

In [25]:
def convert_bb(img, width, height):
    bb = []
    conv_x = (224. / width)
    conv_y = (224. / height)
    bb.append(bb_json[img]['height'] * conv_y)
    bb.append(bb_json[img]['width'] * conv_x)
    bb.append(max(bb_json[img]['x'] * conv_x, 0))
    bb.append(max(bb_json[img]['y'] * conv_y, 0))
    return bb

In [26]:
len(bb_json.keys())

4982

In [27]:
trn_bbox = []
val_bbox = []

In [28]:
for i in range(len(trn_filename)):
    trn_bbox.append(convert_bb(trn_filename[i], trn_size[i][0], trn_size[i][1]))

In [29]:
for i in range(len(val_filename)):
    val_bbox.append(convert_bb(val_filename[i], val_size[i][0], val_size[i][1]))

In [30]:
trn_bbox = np.asarray(trn_bbox)
val_bbox = np.asarray(val_bbox)

In [31]:
print(trn_bbox[-1200])
print(val_bbox[-600])

[ 34.95296  47.12925  60.94725  73.27488]
[ 32.42710472  53.025       94.5         88.08213552]


# NN

In [32]:
input_img = Input(shape=(224, 224, 3))

x = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(input_img)
x = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x)
x = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dense(512, activation = 'relu')(x)
x = Dense(512, activation = 'relu')(x)

x_bb = Dense(4, name='bb')(x)
x_class = Dense(8, activation='softmax', name='class')(x)

model = Model([input_img], [x_bb, x_class])
model.compile(Adam(lr=0.001), loss=['mse', 'categorical_crossentropy'], metrics=['accuracy'],
             loss_weights=[.001, 1.])

In [33]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 224, 224, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 112, 112, 32) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [34]:
model.fit(X_train, [trn_bbox, y_train], batch_size=batch_size, epochs=10, 
             validation_data=(X_valid, [val_bbox, y_valid]))

Train on 3025 samples, validate on 752 samples
Epoch 1/10
3025/3025 [==============================] - 44s 14ms/step - loss: 19.8723 - bb_loss: 16929.7443 - class_loss: 2.9425 - bb_acc: 0.2936 - class_acc: 0.5200 - val_loss: 2.8174 - val_bb_loss: 1683.2037 - val_class_loss: 1.1342 - val_bb_acc: 0.3657 - val_class_acc: 0.5492
Epoch 2/10
3025/3025 [==============================] - 39s 13ms/step - loss: 1.8945 - bb_loss: 1285.1528 - class_loss: 0.6094 - bb_acc: 0.3898 - class_acc: 0.8321 - val_loss: 1.6957 - val_bb_loss: 982.7868 - val_class_loss: 0.7129 - val_bb_acc: 0.4016 - val_class_acc: 0.8378
Epoch 3/10
3025/3025 [==============================] - 39s 13ms/step - loss: 0.9927 - bb_loss: 767.8120 - class_loss: 0.2249 - bb_acc: 0.4317 - class_acc: 0.9412 - val_loss: 1.1554 - val_bb_loss: 689.7268 - val_class_loss: 0.4657 - val_bb_acc: 0.4362 - val_class_acc: 0.9003
Epoch 4/10
3025/3025 [==============================] - 39s 13ms/step - loss: 0.6485 - bb_loss: 520.3297 - class_loss: 0